In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# seed 값 설정 
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# 데이터 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:
# 컨볼루션 신경망 설정
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [5]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [6]:
# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [7]:
modelpath = MODEL_DIR + "mnist-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                              verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [8]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30,
                    batch_size=200, #epoch=5, verbose=2,
                    callbacks=[early_stopping_callback, checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 46s 761us/step - loss: 0.2868 - accuracy: 0.9118 - val_loss: 0.0539 - val_accuracy: 0.9831

Epoch 00001: val_loss improved from inf to 0.05394, saving model to ./model/mnist-cnn-01-0.0539.hdf5
Epoch 2/30
60000/60000 [==============================] - 54s 902us/step - loss: 0.0903 - accuracy: 0.9733 - val_loss: 0.0406 - val_accuracy: 0.9870

Epoch 00002: val_loss improved from 0.05394 to 0.04057, saving model to ./model/mnist-cnn-02-0.0406.hdf5
Epoch 3/30
60000/60000 [==============================] - 45s 742us/step - loss: 0.0690 - accuracy: 0.9791 - val_loss: 0.0347 - val_accuracy: 0.9883

Epoch 00003: val_loss improved from 0.04057 to 0.03469, saving model to ./model/mnist-cnn-03-0.0347.hdf5
Epoch 4/30
60000/60000 [==============================] - 44s 731us/step - loss: 0.0547 - accuracy: 0.9834 - val_loss: 0.0372 - val_accuracy: 0.9876

Epoch 00004: val_loss did not improve f

In [9]:
from keras.models import load_model
del model
model = load_model('model/mnist-cnn-15-0.0282.hdf5')

OSError: Unable to open file (unable to open file: name = 'model/mnist-cnn-15-0.0282.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy : %.4f" % (model.evaluate(x_test, y_test, verbose=0)[1]))

In [ ]:
# 테스트셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

In [ ]:
# 그래프로 표현
x_len = np.arange(1, len(y_loss) + 1)
plt.figure(figsize=(8,6))
plt.plot(x_len, y_vloss, marker='.', c="red", label="Testset_loss")
plt.plot(x_len, y_loss, marker='.', c="blue", label="Trainset_loss")

#그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

### 결론 : Best Model은 Epoch 15, 정확도는 99.18%